# SPPNet (Spatial Pyramid Pooling in Deep Convolutional Networks for Visual Recognition)

- [paper](https://arxiv.org/pdf/1406.4729.pdf)

### Introduction
기존 CNN은 fixed-size ($e.g., 224 \times 224$) input image가 필요하므로, 고정된 벡터 크기를 맞춰주기 위해서 crop, wrap을 적용합니다.\
feature extraction layer는 sliding window를 사용하기 때문에 고정된 사이즈가 필요하지 않지만,\
FC layer는 고정된 입력 크기가 필요하기 때문입니다.

crop과 warp은 classification에서는 data augmentation, detection에서는 region proposal을 입력 사이즈에 맞춰주기 위해 이용합니다.\
여기서 crop과 warp을 적용하면 문제점이 발생합니다.

crop을 적용하면 crop된 구역만 CNN을 통과시키기 때문에, 전체 이미지 정보가 손실이 발생합니다.\
위 그림에서 자동차 사진을 입력 크기로 맞춰주기 위해 자동차가 잘려나간 것을 확인할 수 있습니다.

또, warp을 적용하면 이미지에 변형이 일어납니다.\
등대가 기존의 가로세로비를 유지하지 못하고 옆으로 퍼진 채로 CNN을 통과하게 되는 것입니다.\
이처럼 입력 이미지 크기를 고정시키면 이미지 정보 손실과 변형이 발생합니다.

<img src = "https://github.com/fned49/blog/blob/main/SPPNet/figure/figure1.png?raw=true">

SPPnet은 FC layer 이전에 Spatial Pyramid Pooling layer를 추가하여,\
convolutional layer가 임의의 사이즈로 입력을 취할 수 있게 합니다.\
입력 사이즈 제한을 제거함으로써 입력 이미지를 crop / warp 필요성을 없앤 것입니다.

사실 Fc layer전까지의 CNN 자체는 임의의 사이즈(arbitrary)를 넣어도 괜찮기 때문에,\
본 논문의 핵심은 원본 영상을 왜곡없이 입력으로 넣어 feature들을 압축하면,\
이를 최종적으로 fc layer에 맞게 고정 길이의 벡터로 압축하는 풀링 방법론을 제안합니다.

이 방법이 Object Detection 모델에 크게 유효했습니다.

### SPP layer

<img src = "https://github.com/fned49/blog/blob/main/SPPNet/figure/figure2.png?raw=true">

input image가 고정 사이즈가 아니었기 때문에, 최종 Conv를 거친 feature map은 사이즈가 arbitrary합니다.\
이때 feature map 크기가 어떻든간에 피라미드 풀링을 하면,\
$bin\times256$ 으로 feature를 고정된 사이즈로 압축 가능합니다.

<img src = "https://github.com/fned49/blog/blob/main/SPPNet/figure/figure4.png?raw=true">

위처럼 다양한 사이즈를 동일한 feature map에 적용해서, 전부 Concat하는 구조라 "피라미드"라는 이름을 붙인 것 같습니다.\
figure3 Conv`5에서 나온 feature map에 bin=21인 피라미드 풀링을 적용하면, 위 그림처럼 최종 벡터가 완성됩니다.\
풀링 전 검은색 feature map은 arbitrary size이지만,\
최종적으로 완성되는 벡터는 bin * 256(최종 feature map의 채널)으로 고정될 수 있는 겁니다.

맥스 풀링도 window size랑 stride가 있습니다.\
최종 feature map(그림에서 검은색 feature map)의 사이즈를 axa라고 할 때 적용하려는 피라미드 풀링 사이즈가 n x n이면,\
윈도우 사이즈는 ceiling(a/n), stride는 floor(a/n)입니다. (각각 소수점 올림, 버림)\
빨간색 풀링은 흔히 알고있는 Global max Pooling이 되겠네요.\

<img src = "https://github.com/fned49/blog/blob/main/SPPNet/figure/figure3.png?raw=true">

이렇게 다양한 사이즈로 풀링을 진행하면, 정보가 더욱 다양해지는 효과가 있습니다.\
또한 다양한 사이즈의 객체를 검출하는 데에 유리한 장점이 있습니다.


### 문제점

1. end to end가 아니라서 feature들을 저장하여 알고리즘들을 각각 따로 학습시켜야 한다. 
2. 각각 학습을 하기 때문에 CNN 알고리즘도 svm과 regressor의 결과와 독립적으로 학습되어 아쉽다.